In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob as gb
import numpy as np


In [2]:
path = os.getcwd()+ '/Downloads/car_data'

In [3]:
train = path+"/train"
test = path+"/test"
train_labels = []
train_images = []
test_labels = []
test_images = []


In [4]:
train_path =  [folder for folder in os.listdir(train) if not folder.startswith('.')]
test_path =  [folder for folder in os.listdir(test) if not folder.startswith('.')]


## get # of photos for train & test
train_len = -1
for i in os.walk(train, topdown=True):
        train_len += len(i[2]) 

test_len = -1
for i in os.walk(test, topdown=True):
        test_len += len(i[2]) 

## add the type of cars into dict
d = dict()
idx = 0
for folder in test_path:
    d[folder] = idx
    idx+= 1

In [5]:
for folder in test_path:
    jpg_path = gb.glob(pathname = test + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)
        image = cv2.imread(img)
        test_labels.append(np.array(d[folder]))
        image = cv2.resize(image, (550,550))
        test_images.append(np.array(image))

for folder in train_path:
    jpg_path = gb.glob(pathname = train + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)
        image = cv2.imread(img)
        train_labels.append(np.array(d[folder]))
        image = cv2.resize(image, (550,550))
        train_images.append(np.array(image))
        

In [6]:
train_labels = np.array(train_labels)
train_labels = np.reshape(train_labels, (-1, 1))

test_labels = np.array(test_labels)
test_labels = np.reshape(test_labels, (-1, 1))

train_images = np.array(train_images)
test_images = np.array(test_images)



In [ ]:
train_images, test_images= train_images / 255.0, test_images/ 255.0


In [ ]:
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(550, 550, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
for folder in train_path:
    jpg_path = gb.glob(pathname = test + "/" + folder + "/*.jpg")
    
    for img in jpg_path:
        img_name = os.path.basename(img)
        image = cv2.imread(img)
        np.append(test_labels,np.array(folder))
        np.append(test_images,np.array(image))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

model.summary()

In [ ]:
y_pred = model.predict(test_images)
pred_label = np.argmax(y_pred,axis = 1)
plt.figure(figsize=(25,25))

def getLabel(i):
    return (list(d.keys())[list(d.values()).index(i)])  

for i in range(25):
    
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    plt.imshow(test_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.title("True: " + getLabel(test_labels[i][0]) + "\n" + "Predicted: " + getLabel(pred_label[i]))
plt.tight_layout()
plt.show()